In [1]:
dotenv_path = "../../../../../apis/.env"

import torch
from transformers import pipeline
from IPython.display import clear_output
import pandas as pd
from datasets import Dataset
import random
from textwrap import fill

import sys
sys.path.append("./src")
import text_classification_functions as tcf
import env_options
import lmsys_dataset_handler as lmsys

hf_token, hf_token_write = env_options.check_env(colab=False, use_dotenv=True, dotenv_path=dotenv_path)

Python version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
PyTorch version: 2.2.2
Transformers version: 4.44.2
CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU
CUDA Version: 12.1
FlashAttention available: True
Retrieved HuggingFace token(s) from .env file
Using HuggingFace token: hf_M*****************************IASJ
Using HuggingFace write token: hf_u*****************************Xipx


In [4]:
### DEBUG ###
import importlib
importlib.reload(lmsys)
### DEBUG ###

<module 'lmsys_dataset_handler' from 'c:\\Users\\david\\Documents\\git\\chatbot-arena-wrapper\\./src\\lmsys_dataset_handler.py'>

### In-memory sampling

In [36]:
N_SAMPLES = 100 # Number of full conversations to extract from the dataset: use a high number if streaming (samples chosen at random only if storing locally)
MIN_CHAR_LENGTH = 50
MAX_CHAR_LENGTH = 500 # Maximum character length of the prompts to be labeled
exclusions = 'prompts/exclusions.txt'
# exclusions = None

lmsys_chat_1m = lmsys.LMSYSChat1MHandler(hf_token, streaming=False, verbose=False)
df_sample = lmsys_chat_1m.parquet_sampling(n_samples=N_SAMPLES)
# df_sample = lmsys_chat_1m.extract_df_sample(N_SAMPLES) # Slower
df_prompts = lmsys_chat_1m.extract_prompts(filter_language=[], 
                                           min_char_length= MIN_CHAR_LENGTH,
                                           max_char_length=MAX_CHAR_LENGTH,
                                           exclusions=exclusions)
# df_prompts.to_csv("csv/orig_sample.csv", encoding='utf-8')

prompt_sample = lmsys_chat_1m.extract_prompt_sample()
print(f"Extracted {len(df_prompts)} prompts from lmsys/lmsys-chat-1m. Prompt sample:\n")
print(prompt_sample)

Sampling from train-00002-of-00006-1779b7cec9462180.parquet
Retrieved 100 random conversations from lmsys/lmsys-chat-1m/train-00002-of-00006-1779b7cec9462180.parquet
Excluded 8 prompts.
Extracted 96 prompts from lmsys/lmsys-chat-1m. Prompt sample:

Please act as a Chinese-English translation coach. You first randomly generate a sentence in Chinese about any topic, and use 'question:' as a prompt. You will then ask the user to enter the English translation of the sentence. When the user enters the English translation of the sentence, you will evaluate the user's answer and provide the correct English translation of the Chinese sentence.Finally, please repeat the above three steps.


### Print random conversation from sample

In [37]:
# Showing an example of a multi-turn conversation
df_sample_with_turns = lmsys_chat_1m.add_turns_to_conversations()
multi_turn_conversation_indices = df_sample_with_turns[df_sample_with_turns['turn'] > 1].index
random_conversation_index = random.choice(multi_turn_conversation_indices)
print(f"\nConversation ID {df_sample_with_turns.loc[random_conversation_index, 'conversation_id']}:\n")
#print(df_sample_with_turns.loc[random_conversation_index, 'conversation'])
conversation = df_sample_with_turns.loc[random_conversation_index, 'conversation']
for turn in conversation:
    user = turn.get('role')
    content = turn.get('content', '')
    wrapped_content = fill(content, width=130)
    role = '😎' if user == 'user' else '🤖'
    print(f"{role} {wrapped_content}\n")


Conversation ID c92f1e63f567400ba37f87f5807d6e6d:

😎 hi

🤖 Hello! How can I assist you today?

😎 are u gpt4?

🤖 No, I am not GPT-4. I am a language model trained on a large dataset of text, but I am not the latest version of GPT. GPT-4, if it
exists, is a future development and not yet publicly available.

😎 boring

🤖 I'm sorry if my response was not interesting enough. Is there something specific you would like to know or discuss? I'll do my
best to provide a more engaging response.

😎 u're not as good as gpt

🤖 As a language model, I strive to provide the most accurate and informative responses to your questions to the best of my
abilities. However, I acknowledge that there are limitations to my abilities and I may not always be able to provide the exact
information you are looking for. If you have any specific questions, I will do my best to provide a helpful response.

😎 just downvoted u

🤖 I'm sorry that my response did not meet your expectations. As an AI language model, I don't